<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/workflow_notebooks/update_all_matchs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")
install_package("tqdm")

In [8]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas

# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

# Games Update

In [17]:
from tqdm import tqdm
import re

df_all_matchs=get_file_content("data/df_all_matchs.csv")
df_incoming_matchs=get_file_content("data/df_incoming_matchs.csv")
df_tournaments=get_file_content("data/df_tournaments.csv")

df_tournaments=df_tournaments[(pd.to_datetime(df_tournaments['tournament_date'])>=(datetime.now()-timedelta(days=15)))&
                              (pd.to_datetime(df_tournaments['tournament_date'])<=(datetime.now()+timedelta(days=15)))]

list_matchs=[]
list_matchs_incoming=[]
pct_tournaments=1
nb_rows=int(pct_tournaments*len(df_tournaments))
for index,row in tqdm(df_tournaments[:int(nb_rows)].iterrows(), total=nb_rows):
  tournament_url=row["url"]
  tournament_date=row["tournament_date"]
  tournament_name=row["tournament_name"]
  find_matchs_all=find_matchs(tournament_url,tournament_date,tournament_name)
  list_matchs+=find_matchs_all[0]
  list_matchs_incoming+=find_matchs_all[1]

result = pd.merge(pd.DataFrame(list_matchs), df_all_matchs, how='left', indicator=True)
rows_not_in_df_match = result[result['_merge'] == 'left_only'].drop(columns=['_merge'])

final_df=pd.concat([df_all_matchs,rows_not_in_df_match]).reset_index(drop=True)
incoming_games=pd.DataFrame(list_matchs_incoming)
incoming_games["date_added"]=datetime.now().strftime('%d-%m-%Y')

df_incoming_matchs=pd.concat([df_incoming_matchs,incoming_games]).reset_index(drop=True)
df_incoming_matchs=df_incoming_matchs.drop_duplicates(["tournament_name","tournament_date","round",	"player_1_name","player_2_name"])



add_file_github('data/df_all_matchs.csv',final_df.to_csv(index=False))
add_file_github('data/df_incoming_matchs.csv',df_incoming_matchs.to_csv(index=False))

100%|██████████| 6/6 [00:08<00:00,  1.34s/it]


In [ ]:
# matchs_report=get_file_content("update_all_matchs_report.txt")

# number_new_tournaments=len(final_df['tournament_name'].unique())-len(df_all_matchs['tournament_name'].unique())
# number_new_games=len(final_df)-len(df_all_matchs)
# time_run=datetime.now()

# string_return=str(time_run)+': '+str(number_new_tournaments)+' new tournaments and '+str(number_new_games)+' new games. Total Number of games: '+str(len(final_df))+' - Total number of players: '+str(len(final_df['loser_name'].unique()))
# matchs_report.append(string_return)

# add_file_github('update_all_matchs_report.txt',str(matchs_report).replace(',',',\n'))

create_report_workflow(df_all_matchs,final_df,'reports/update_all_matchs_report.json',['loser_name'])
